In [1]:
## For platforms
import os


def get_platform():
    platform = ''

    # Windows
    if os.name == 'nt':
        try:
            get_ipython().__class__.__name__
            platform = 'jupyter'
        except NameError:
            platform = 'python'

    elif os.name == 'posix':
        # Kaggle
        if 'KAGGLE_DATA_PROXY_TOKEN' in os.environ.keys():
            platform = 'kaggle'

    # Google Colab
        else:
            try:
                from google.colab import drive
                platform = 'colab'
            except ModuleNotFoundError:
                platform = None

    print(f'Use: {platform}')
    return platform


def colab_action():
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    os.chdir(f'/content/drive/My Drive/LOB/Pipeline')
    os.system('pip install automodinit keras_tuner')
    os.system('nohup /usr/bin/python3 Colab_saver.py &')


def kaggle_action():
    ...


platform = get_platform()
if platform == 'colab':
    colab_action()
elif platform == 'kaggle':
    kaggle_action()
    

import backend as B
B.set_backend(platform)

import numpy as np
import pandas as pd
import tensorflow as tf

from backend import DataBack,ModelBack,DataClass
seq_len = 100

Use: jupyter
Using TensorFlow backend
Dataset  : ../dataset/saved_data
Callbacks: ../Temp/callbacks


In [2]:
from models import m_preln as test_model

In [3]:
## Load data
proportion = input('Data proportion 100-0 in % (press enter for all): ')
if proportion == '': proportion = 1
else: proportion = float(proportion) / 100

train, val, test = DataBack.from_saved(proportion=proportion,
                                       train_indexes=[6],
                                       val_indexes=[6])
DataBack.inspect_data(train=train, val=val, test=test)

ds_train = DataBack.build_dataset(data=train, seq_len=seq_len, batch_size=100)
ds_val = DataBack.build_dataset(data=val, seq_len=seq_len, batch_size=100)
DataBack.inspect_dataset(train=ds_train, val=ds_val)

    Datas:
train     : x= (31608, 40)     | y= (31608,)       
train     : x= (62326, 40)     | y= (62326,)       
train     : x= (85154, 40)     | y= (85154,)       
train     : x= (114772, 40)    | y= (114772,)      
train     : x= (142600, 40)    | y= (142600,)      
train     : x= (173922, 40)    | y= (173922,)      
train     : x= (203799, 40)    | y= (203799,)      
train     : x= (248181, 40)    | y= (248181,)      
train     : x= (289919, 40)    | y= (289919,)      
val       : x= (7903, 40)      | y= (7903,)        
val       : x= (15582, 40)     | y= (15582,)       
val       : x= (21289, 40)     | y= (21289,)       
val       : x= (28694, 40)     | y= (28694,)       
val       : x= (35651, 40)     | y= (35651,)       
val       : x= (43481, 40)     | y= (43481,)       
val       : x= (50950, 40)     | y= (50950,)       
val       : x= (62046, 40)     | y= (62046,)       
val       : x= (72480, 40)     | y= (72480,)       
    Datasets:
train : [13519, 100, 40]
val   : [3375,

In [4]:
PARAMETRS= DataClass(test_model.PARAMETRS)
PARAMETRS

convolutional
├─ dilation_steps: 4
└─ filters: 14
feed_forward
├─ activation: function relu
├─ dropout_rate: 0.1
├─ kernel_initializer: glorot_uniform
├─ kernel_regularizer: keras.regularizers.L2 object
├─ out_activation: softmax
└─ units: 64
optimizer: keras.optimizers.legacy.adam.Adam object
seq_len: 100
transformer
├─ blocks: 2
├─ heads: 3
└─ share_weights: False

In [5]:
## Set up parametrs

In [6]:
## Build
tf.keras.backend.clear_session()
restore = True if input('Restore? (y-yes, enter-no): ') == 'y' else False
input_name = ''
while input_name == '':
    input_name = input(
        f"Input train name to {'restore' if restore else 'build new'}: ")
if restore:
    model, train_name = ModelBack.restore_model(input_name)
else:
    model = test_model.blocks.build_model(**PARAMETRS.DATA_NESTED)
    train_name = ModelBack.get_training_name(input_name)
    print(
        f'Pattern model: {test_model.__name__}',
        f'Train name: {train_name}',
        'Parametrs:',
        PARAMETRS,
        sep='\n',
    )

model.summary()

Pattern model: models.m_preln
Train name: full1(20;05;57--15.10)
Parametrs:
convolutional
├─ dilation_steps: 4
└─ filters: 14
feed_forward
├─ activation: function relu
├─ dropout_rate: 0.1
├─ kernel_initializer: glorot_uniform
├─ kernel_regularizer: keras.regularizers.L2 object
├─ out_activation: softmax
└─ units: 64
optimizer: keras.optimizers.legacy.adam.Adam object
seq_len: 100
transformer
├─ blocks: 2
├─ heads: 3
└─ share_weights: False

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 40)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 14)           1134      
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_2 (

In [7]:
## Callbacks
callback_freq = 'epoch'
train_dir = f'{ModelBack.callback_path}/{train_name}'
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=train_dir,
        histogram_freq=1,
        update_freq=callback_freq,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f'{train_dir}/checkpoints/' + '{epoch:04d}.keras',
        monitor="val_sp_acc",
        verbose=0,
        save_best_only=False,
        save_weights_only=False,
        mode="auto",
        save_freq=callback_freq,
    )
]
to_dump = PARAMETRS
to_dump.desc = input(f"Input description: ")

ModelBack.dump_data(to_dump, model_path=train_dir)
print(
    f"Callbacks:\n{[str(type(callback)).split('.')[-1] for callback in callbacks]}",
    f'Directory: {train_dir}',
    sep='\n',
)


Callbacks:
["TensorBoard'>", "ModelCheckpoint'>"]
Directory: ../Temp/callbacks/full1(20;05;57--15.10)


In [8]:
# %tensorboard

In [9]:
## Train
training_question = ''
while training_question not in ['y', 'n']:
    training_question = input(f'Start training now (y-yes) (n-exit): ')
if training_question == 'y':
    model.fit(
        ds_train,
        epochs=20,
        validation_data=ds_val,
        callbacks=callbacks,
    )

Epoch 1/20
